In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
import os

from sklearn.model_selection import train_test_split

import tensorflow as tf

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [2]:
train_images = os.listdir('petfinder-pawpularity-score/train')
train_df = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_df = train_df.set_index('Id')

file_names = []
labels = []

for index in train_df.index:
    file_name = 'petfinder-pawpularity-score/train/' + index + '.jpg'
    label = train_df.loc[index]['Pawpularity']
    file_names.append(file_name)
    labels.append(label/1.0)



In [3]:
train_files, val_files, labels_train, labels_val = train_test_split(file_names, labels, test_size=0.2, random_state=125)

In [4]:
def process_file_names_train(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.image.resize(img, [224,224])
    img = img/255.
    # drop this for val data
    img = tf.image.resize_with_crop_or_pad(img, 224 + 6, 224 + 6)
    # Random crop back to the original size.
    image = tf.image.random_crop(
      img, size=[224, 224, 3])
    # Random brightness.
    image = tf.image.random_brightness(
      image, max_delta=0.5)
    image = tf.clip_by_value(image, 0, 1)
    # rescale labels so between 0 and 1
    return image, label/100.01

def process_file_names_val(file_name, label):
    image_string = tf.io.read_file(file_name)
    img = tf.io.decode_jpeg(image_string, channels=3)
    img = tf.image.resize(img, [224,224])
    img = img/255.
    # rescale labels so between 0 and 1
    return img, label/100.01

train_dataset = tf.data.Dataset.from_tensor_slices((train_files, labels_train))
train_dataset = train_dataset.map(process_file_names_train)

val_dataset = tf.data.Dataset.from_tensor_slices((val_files, labels_val))
val_dataset = val_dataset.map(process_file_names_val)

2024-03-25 10:49:41.349923: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-25 10:49:41.349941: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-25 10:49:41.349945: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-25 10:49:41.349974: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-25 10:49:41.349988: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(32)
    return ds

train_dataset = configure_for_performance(train_dataset)

val_dataset = configure_for_performance(val_dataset)

In [25]:
def build_cnn():
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=64, kernel_size=5, activation="relu", padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Conv2D(
        filters=128, kernel_size=5, activation="relu", padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Conv2D(
        filters=256, kernel_size=5, activation="relu", padding="same"))
    #model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.GlobalAvgPool2D())
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(units=512, activation="relu"))
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.003),
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model
CNN = build_cnn()
# use early stopping
#early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
# Train the model
history = CNN.fit(train_dataset,
                  epochs=10,
                  validation_data=val_dataset,
                  verbose=1)
                  #callbacks=[early_stopping_callback])
#test_loss, test_metric = CNN.evaluate(images_test, labels_test)

Epoch 1/10
248/248 [==============================] - 234s 922ms/step - loss: 0.6950 - root_mean_squared_error: 0.2101 - val_loss: 0.6648 - val_root_mean_squared_error: 0.2128
Epoch 2/10
248/248 [==============================] - 231s 930ms/step - loss: 0.6644 - root_mean_squared_error: 0.2044 - val_loss: 0.6648 - val_root_mean_squared_error: 0.2127
Epoch 3/10
248/248 [==============================] - 228s 918ms/step - loss: 0.6644 - root_mean_squared_error: 0.2045 - val_loss: 0.6647 - val_root_mean_squared_error: 0.2127
Epoch 4/10
  6/248 [..............................] - ETA: 3:32 - loss: 0.6621 - root_mean_squared_error: 0.2104

KeyboardInterrupt: 

In [26]:
CNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     204928    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 256)       819456    
                                                                 
 global_average_pooling2d (  (None, 256)               0

# ResNet

In [6]:
# define residual layer

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            tf.keras.layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same', kernel_initializer='he_normal', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            tf.keras.layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal', use_bias=False),
            tf.keras.layers.BatchNormalization()
        ]
        # If strides > 1 need to use 1D convolutions with long stride to resize image so they can be added
        if strides == 1:
            self.skip_layers = []
        else:
            self.skip_layers = [
                tf.keras.layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same', kernel_initializer='he_normal', use_bias=False),
                tf.keras.layers.BatchNormalization()
            ]
    def call(self, inputs):
        z = inputs
        for layer in self.main_layers:
            z = layer(z)
        skip_z = inputs
        for layer in self.skip_layers:
            skip_z = layer(skip_z)
        # either return a(z) + skip_z or a(z + skip_z)
        return self.activation(z + skip_z)

In [7]:
def build_resnet():
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_initializer='he_normal', use_bias=False, input_shape=[224,224,3]))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'))
    prev_filters = 128
    for filters in [64, 64, 128, 256, 512]:
        strides = 1 if filters == prev_filters else 2
        model.add(ResidualUnit(filters, strides=strides))
        prev_filters = filters


    model.add(tf.keras.layers.GlobalAvgPool2D())
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(units=512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(),
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model
resnet = build_resnet()
# use early stopping
#early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
# Train the model

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
history = resnet.fit(train_dataset,
                  epochs=10,
                  validation_data=val_dataset,
                  verbose=1,
                  callbacks=[early_stopping_callback])
#test_loss, test_metric = CNN.evaluate(images_test, labels_test)

Epoch 1/10


2024-03-25 10:49:55.365470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


248/248 [==============================] - 54s 194ms/step - loss: 0.7000 - root_mean_squared_error: 0.2373 - val_loss: 0.9065 - val_root_mean_squared_error: 0.3544
Epoch 2/10
248/248 [==============================] - 43s 173ms/step - loss: 0.6744 - root_mean_squared_error: 0.2181 - val_loss: 0.6829 - val_root_mean_squared_error: 0.2202
Epoch 3/10
248/248 [==============================] - 42s 169ms/step - loss: 0.6685 - root_mean_squared_error: 0.2123 - val_loss: 0.6670 - val_root_mean_squared_error: 0.2011
Epoch 4/10
248/248 [==============================] - 42s 170ms/step - loss: 0.6674 - root_mean_squared_error: 0.2111 - val_loss: 0.6894 - val_root_mean_squared_error: 0.2233
Epoch 5/10
248/248 [==============================] - 43s 172ms/step - loss: 0.6654 - root_mean_squared_error: 0.2089 - val_loss: 0.6657 - val_root_mean_squared_error: 0.1998
Epoch 6/10
248/248 [==============================] - 39s 159ms/step - loss: 0.6629 - root_mean_squared_error: 0.2062 - val_loss: 0.6675

In [8]:
labels_train_array = np.array(labels_train)
labels_val_array = np.array(labels_val)
train_mean = np.mean(labels_train_array)

print(np.sqrt(np.mean(np.square(labels_train_array - train_mean))))
print(np.sqrt(np.mean(np.square(labels_val_array - train_mean))))

20.755687461850368
19.91888946933486


In [16]:
np.max(labels)

100.0

In [35]:
resnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      9408      
                                                                 
 batch_normalization (Batch  (None, 112, 112, 64)      256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)        0         
 D)                                                              
                                                                 
 residual_unit (ResidualUni  (None, 28, 28, 64)        78592     
 t)                                                              
                                                        

In [38]:
def build_resnet():
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_initializer='he_normal', use_bias=False, input_shape=[224,224,3]))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'))
    prev_filters = 128
    for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
        strides = 1 if filters == prev_filters else 2
        model.add(ResidualUnit(filters, strides=strides))
        prev_filters = filters


    model.add(tf.keras.layers.GlobalAvgPool2D())
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(units=512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model
resnet = build_resnet()
# use early stopping
#early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
# Train the model
history = resnet.fit(train_dataset,
                  epochs=10,
                  validation_data=val_dataset,
                  verbose=1)
                  #callbacks=[early_stopping_callback])
#test_loss, test_metric = CNN.evaluate(images_test, labels_test)

Epoch 1/10
248/248 [==============================] - 72s 275ms/step - loss: 0.7058 - root_mean_squared_error: 0.2316 - val_loss: 0.6884 - val_root_mean_squared_error: 0.2353
Epoch 2/10
248/248 [==============================] - 65s 263ms/step - loss: 0.6811 - root_mean_squared_error: 0.2221 - val_loss: 0.6754 - val_root_mean_squared_error: 0.2247
Epoch 3/10
248/248 [==============================] - 66s 264ms/step - loss: 0.6737 - root_mean_squared_error: 0.2146 - val_loss: 0.7620 - val_root_mean_squared_error: 0.2849
Epoch 4/10
248/248 [==============================] - 66s 265ms/step - loss: 0.6741 - root_mean_squared_error: 0.2149 - val_loss: 0.7315 - val_root_mean_squared_error: 0.2667
Epoch 5/10
248/248 [==============================] - 65s 264ms/step - loss: 0.6705 - root_mean_squared_error: 0.2112 - val_loss: 0.7141 - val_root_mean_squared_error: 0.2547
Epoch 6/10
248/248 [==============================] - 67s 268ms/step - loss: 0.6684 - root_mean_squared_error: 0.2087 - val_l

In [1]:
resnet.summary()

NameError: name 'resnet' is not defined

In [14]:
def vgg16(input_shape=(224, 224, 3)):
    tf.keras.backend.clear_session()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
        #tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create VGG16 model
vgg16_model = vgg16()

# Display model summary
vgg16_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 64)      1792      
                                                                 
 conv2d_4 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 112, 112, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_6 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                 